**Created on 11/01/2022**

**Group 2**

**Authors: Paul Fiche**

**License: this code is released under the CeCILL 2.1 license. See https://www.cecill.info/licenses/Licence_CeCILL_V2.1-en.txt**

In [ ]:
import os
import sys
import math
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn import tree
from sklearn.feature_selection import SequentialFeatureSelector

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def score_distanceM(Xobserve:list,Yobserve:list,Xprevu:list,Yprevu:list)-> float:
  X=(Xobserve-Xprevu)**2
  Y=(Yobserve-Yprevu)**2
  return np.mean(np.sqrt(X+Y))

In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


opening the dataset

In [ ]:
test=pd.read_csv('test_tableau_31.csv')
train=pd.read_csv('train_tableau_31.csv')

removing target and seqnum variables

In [ ]:
variable_sans_target=test.columns.tolist()
del variable_sans_target[variable_sans_target.index('Unnamed: 0')]
del variable_sans_target[variable_sans_target.index('target')]
del variable_sans_target[variable_sans_target.index('target2')]
del variable_sans_target[variable_sans_target.index('target3')]
del variable_sans_target[variable_sans_target.index('seqnum')]
del variable_sans_target[variable_sans_target.index('seqnum2')]
del variable_sans_target[variable_sans_target.index('seqnum3')]

In [ ]:
test=test[variable_sans_target]
train=train[variable_sans_target]

data distribution

In [ ]:
variable=train.columns.tolist()
del variable[variable.index('loc_initiator_x')]
del variable[variable.index('loc_initiator_y')]
X=train[variable]
X_test=test[variable]
y_test=test[['loc_initiator_x','loc_initiator_y']]

first test of the decision tree without variable selection

In [ ]:
y=train[['loc_initiator_x','loc_initiator_y']]

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.33, random_state=42)

clf=tree.DecisionTreeRegressor()
clf= clf.fit(X_train, y_train)

scores = cross_val_score(clf, X, y, cv=5)

y_predict=clf.predict(X_validation)

print('validation croisee :',scores)
print()
print( 'score : ', clf.score(X_validation,y_validation) )


print()
print("RMSE sur  validation: % f" %(mean_squared_error(y_predict,y_validation)))
print()
print("MAE sur  validation: % f" %(mean_absolute_error(y_predict,y_validation)))
print()
print('score distance validation : ',score_distanceM(y_validation.loc_initiator_x, y_validation.loc_initiator_y,y_predict[:,0],y_predict[:,1]))
y_predict=clf.predict(X_test)

print()
print("RMSE sur  test: % f" %(mean_squared_error(y_predict,y_test)))
print()
print("MAE sur  test: % f" %(mean_absolute_error(y_predict,y_test)))

print('score distance test : ',score_distanceM(y_test.loc_initiator_x, y_test.loc_initiator_y,y_predict[:,0],y_predict[:,1]))


validation croisee : [0.99664731 0.99644903 0.9962282  0.99646208 0.99651508]

score :  0.9958082450966995

RMSE sur  validation:  0.099019

MAE sur  validation:  0.094157

score distance validation :  0.14109085761716242

RMSE sur  test:  2.035351

MAE sur  test:  0.630918
score distance test :  0.9475991319607061


The average error distance is 0.94m

##Selection de feature

In [ ]:
list_variable=variable
clf = tree.DecisionTreeRegressor()
sfs = SequentialFeatureSelector(clf, direction='backward',cv=3)
sfs.fit(X_train, y_train)
target_opt=[]
bool_choix=sfs.get_support()
for i in range(len(bool_choix)):
  if bool_choix[i]:
    target_opt.append(list_variable[i])

In [ ]:
print(target_opt)

['nlos_indicator', 'skew', 'tof', 'loc_target_x', 'loc_target_y', 'rssi_data', 'nlos_indicator2', 'skew2', 'range2', 'loc_target_x2', 'loc_target_y2', 'nlos_indicator3', 'tof3', 'range_skew3', 'loc_target_x3', 'rssi_request3', 'rssi_data3', 'orientation3', 'origin', 'rotation']


the selected features are : ['nlos_indicator', 'skew', 'tof', 'loc_target_x', 'loc_target_y', 'rssi_data', 'nlos_indicator2', 'skew2', 'range2', 'loc_target_x2', 'loc_target_y2', 'nlos_indicator3', 'tof3', 'range_skew3', 'loc_target_x3', 'rssi_request3', 'rssi_data3', 'orientation3', 'origin', 'rotation']

Testing the model after feature selection

In [ ]:
X=train[target_opt]
X_test=test[target_opt]
y_test=test[['loc_initiator_x','loc_initiator_y']]
y=train[['loc_initiator_x','loc_initiator_y']]


X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.33, random_state=42)

clf=tree.DecisionTreeRegressor()
clf= clf.fit(X_train, y_train)

scores = cross_val_score(clf, X, y, cv=5)

y_predict=clf.predict(X_validation)

print('cross_val_score :',scores)
print()
print( 'score : ', clf.score(X_validation,y_validation) )


print()
print("RMSE sur  validation: % f" %(mean_squared_error(y_predict,y_validation)))
print()
print("MAE sur  validation: % f" %(mean_absolute_error(y_predict,y_validation)))
print()
print('score distance validation : ',score_distanceM(y_validation.loc_initiator_x, y_validation.loc_initiator_y,y_predict[:,0],y_predict[:,1]))
y_predict=clf.predict(X_test)

print()
print("RMSE sur  test: % f" %(mean_squared_error(y_predict,y_test)))
print()
print("MAE sur  test: % f" %(mean_absolute_error(y_predict,y_test)))

print('score distance test : ',score_distanceM(y_test.loc_initiator_x, y_test.loc_initiator_y,y_predict[:,0],y_predict[:,1]))

validation croisee : [0.99690804 0.99644916 0.99683247 0.99697377 0.99719655]

score :  0.9965513747172567

RMSE sur  validation:  0.082977

MAE sur  validation:  0.090596

score distance validation :  0.13587308861323572

RMSE sur  test:  1.272337

MAE sur  test:  0.491778
score distance test :  0.7433733219902908


After feature selection we have an average error distance of 0.74m


##Selection of optimum parameters

In [ ]:
param_grid = {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
              'max_features':['auto', 'sqrt', 'log2','None']
              }

# create an instance of GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5)

# fit the grid search to the data
grid_search.fit(X_train, y_train)

# print the best parameters
print("Best parameters found: ", grid_search.best_params_)
print("Best parameters score: ", grid_search.best_score_)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
35 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 1315, in fit
    super().fit(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 281, in fit
    raise ValueError(
ValueError: Invalid value for max_features. Allowed string values are 'auto', 'sqrt' or 'log2'.


Best parameters found:  {'criterion': 'squared_error', 'max_features': 'auto'}
Best parameters score:  0.9964521215329507


In [ ]:
X=train[target_opt]
X_test=test[target_opt]
y_test=test[['loc_initiator_x','loc_initiator_y']]
y=train[['loc_initiator_x','loc_initiator_y']]


X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.33, random_state=42)

clf=tree.DecisionTreeRegressor(criterion='squared_error',max_features='auto')
clf= clf.fit(X_train, y_train)

scores = cross_val_score(clf, X, y, cv=5)

y_predict=clf.predict(X_validation)

print('cross_val_score :',scores)
print()
print( 'score : ', clf.score(X_validation,y_validation) )


print()
print("RMSE sur  validation: % f" %(mean_squared_error(y_predict,y_validation)))
print()
print("MAE sur  validation: % f" %(mean_absolute_error(y_predict,y_validation)))
print()
print('score distance validation : ',score_distanceM(y_validation.loc_initiator_x, y_validation.loc_initiator_y,y_predict[:,0],y_predict[:,1]))
y_predict=clf.predict(X_test)

print()
print("RMSE sur  test: % f" %(mean_squared_error(y_predict,y_test)))
print()
print("MAE sur  test: % f" %(mean_absolute_error(y_predict,y_test)))

print('score distance test : ',score_distanceM(y_test.loc_initiator_x, y_test.loc_initiator_y,y_predict[:,0],y_predict[:,1]))

validation croisee : [0.99702554 0.99655626 0.99684563 0.99722526 0.99702595]

score :  0.996520034169056

RMSE sur  validation:  0.083290

MAE sur  validation:  0.089787

score distance validation :  0.13492581063764444

RMSE sur  test:  1.281078

MAE sur  test:  0.491169
score distance test :  0.7422029410954255


The score has improved, the average error distance is 0.74 m

#Last test on the whole dataset

In [ ]:
def split(df,num_capteur):
   test=df[(df['target']==num_capteur)|(df['target2']==num_capteur)|(df['target3']==num_capteur)]
   train=df[(df['target']!=num_capteur)&(df['target2']!=num_capteur)&(df['target3']!=num_capteur)]
   return(test,train)

In [ ]:
df=pd.read_csv('complet_v2.csv')
liste_capteur=df.target.unique()
MAE=[]
RMSE=[]
mean_dist=[]
for i in liste_capteur:
  test,train=split(df,i)
  X=train[target_opt]
  X_test=test[target_opt]
  y_test=test[['loc_initiator_x','loc_initiator_y']]
  y=train[['loc_initiator_x','loc_initiator_y']]
  clf=tree.DecisionTreeRegressor(criterion='squared_error',max_features='auto')
  clf= clf.fit(X, y)
  y_predict=clf.predict(X_test)
  RMSE.append(mean_squared_error(y_predict,y_test))
  MAE.append(mean_absolute_error(y_predict,y_test))
  mean_dist.append(score_distanceM(y_test.loc_initiator_x, y_test.loc_initiator_y,y_predict[:,0],y_predict[:,1]))


In [ ]:
df_final=pd.DataFrame({'target':liste_capteur,'MAE':MAE,'RMSE':RMSE,'mean_dist':mean_dist})

In [ ]:
df_final

,target,MAE,RMSE,mean_dist
0,184.0,0.828399,4.637785,1.314304
1,180.0,0.852413,3.142882,1.321900
2,100.0,1.365222,8.682892,2.082386
3,101.0,0.962734,3.742541,1.494385
4,171.0,0.481488,1.458683,0.721338
5,150.0,3.158024,25.935063,4.824079
